In [56]:
#inititalize the notebook
import os
import textwrap
import pymongo
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
)

mongo_user = os.getenv("COSMOS_MONGO_USER")
mongo_pwd = os.getenv("COSMOS_MONGO_PWD")
mongo_server = os.getenv("COSMOS_MONGO_SERVER")
mongo_conn = f"mongodb+srv://{mongo_user}:{mongo_pwd}@{mongo_server}?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"


embeddings_deployment_name = "text-embedding-ada-002"
chat_deployment_name = "gpt-4"

print("*** init done! ***")


*** init done! ***


Hoe ziet een vector er uit?

In [57]:
word = "cat"

embeddings = openai_client.embeddings.create(input = word, model=embeddings_deployment_name)

embeddingData = embeddings.data[0].embedding

print("De vector voor de het woord: " + word)
print("Het aantal dimensies in de vector is: " + str(len(embeddingData)))
print("Dit zijn de waardes van de vector: ")
print(embeddingData)

De vector voor de het woord: cat
Het aantal dimensies in de vector is: 1536
Dit zijn de waardes van de vector: 
[-0.0070945825427770615, -0.017328109592199326, -0.009644086472690105, -0.03070768155157566, -0.012548675760626793, 0.003105211304500699, -0.005113212391734123, -0.04121817275881767, -0.014629469253122807, -0.021376069635152817, 0.019231360405683517, 0.05087646469473839, -0.0012907310156151652, 0.0024855893570929766, -0.03840590640902519, -0.006089693866670132, 0.0355084203183651, -0.004697763826698065, 0.0023630852811038494, -0.01342928409576416, -0.01891888678073883, 0.009019138291478157, 0.015893569216132164, -0.008713766001164913, -0.014672079123556614, 0.007233065087348223, 0.013031589798629284, -0.013365369290113449, 0.002858427818864584, 0.004861102905124426, 0.0040266546420753, -0.01677417755126953, -0.015850959345698357, -0.04306461289525032, -0.027242060750722885, -0.004278764594346285, 0.0080533092841506, -0.009984967298805714, 0.022015219554305077, -0.009040444158

Vergelijk de semantische gelijkheid van woorden

We hebben een lijst van woorden en een woord dat we willen vergelijken.

* Bereken een vector van ieder woord in de lijst
* Bereken een vector van het woord dat we willen vergelijken
* Bepaal de gelijkheid (score) met een dotproduct algoritme (er zijn meerdere algoritmes om dit te doen)

In [58]:
### DotProduct is a simple algorithm to calculate simularity between two vectors
### Multiply each element in vector1 with the corresponding element in vector2
### Add the results together for each element
### The result is the dot product of the two vectors
### https://www.mathsisfun.com/algebra/vectors-dot-product.html

def dotproduct(vector1, vector2):
    #return sum((a*b) for a, b in zip(vector1, vector2))
    #get the length of the vector1 
    length = len(vector1)

    #check if the length of the vector2 is the same as vector1
    if length != len(vector2):
        print("The vectors are not the same length")
        return

    #iterate over the vector based on the length of the vector
    sum = 0

    for i in range(length):
        #multiply the vectors and add them to the sum
        sum += vector1[i] * vector2[i]

    return sum

##### end of the dotproduct function #####

words = ['cat', 
         'hamburger', 
         'computer', 
         'server',
         'dog', 
         'pizza', 
         'laptop', 
         'horse', 
         'car',
         'steak', 
         'truck',
         'pickup']

word = 'food'

#create a list of embeddings for each word
wordListEmbedding = []
for w in words:
    response = openai_client.embeddings.create(input = w, model=embeddings_deployment_name).data[0].embedding
    wordListEmbedding.append(response)

#create an embedding for the word we want to compare
wordEmbedding = openai_client.embeddings.create(input = word, model=embeddings_deployment_name).data[0].embedding

#calculate the dot product between the word we want to compare and each word in the list
dotProductList = []
for w in wordListEmbedding:
    dotProductList.append(dotproduct(wordEmbedding, w))

#sort the dotProductList
sortedDotProductList = sorted(dotProductList, reverse=True)

#find the index of the words with the highest dot product
indexList = []
for i in sortedDotProductList:
    indexList.append(dotProductList.index(i))

#display the top 3 words based on indexlist
print("De top 3 woorden vergelijkbaar met '" + word + "' zijn: ")
print(f"'{words[indexList[0]]}' dotproduct score: + {str(sortedDotProductList[0])}")
print(f"'{words[indexList[1]]}' dotproduct score: + {str(sortedDotProductList[1])}")
print(f"'{words[indexList[2]]}' dotproduct score: + {str(sortedDotProductList[2])}")






De top 3 woorden vergelijkbaar met 'food' zijn: 
'pizza' dotproduct score: + 0.8752008515911022
'steak' dotproduct score: + 0.8483971956644922
'hamburger' dotproduct score: + 0.8465710413055575


De vector kun je ook berekenen over een zin.

In dit geval nemen we user prompt uit het laatste voorbeeld dat we hebben gezien bij "prompts"

In [59]:
sentence = "Wat is er speciaal aan de vrijdag voor de opleiding HBO-ICT in Heerlen?"

embeddings = openai_client.embeddings.create(input = sentence, model=embeddings_deployment_name)

embeddingData = embeddings.data[0].embedding

print("De vector voor de de zin: " + sentence)
print("Het aantal dimensies in de vector is: " + str(len(embeddingData)))
print("Dit zijn de waardes van de vector: ")
print(embeddingData)

De vector voor de de zin: Wat is er speciaal aan de vrijdag voor de opleiding HBO-ICT in Heerlen?
Het aantal dimensies in de vector is: 1536
Dit zijn de waardes van de vector: 
[0.013084392063319683, -0.031345877796411514, 0.0026754748541861773, -0.05081792548298836, -0.0031439245212823153, -0.0010745363542810082, 0.010553798638284206, -0.01497750822454691, 0.006651662290096283, -0.026297567412257195, -0.005202848929911852, 0.004481662064790726, -0.0029668472707271576, -0.01868646964430809, -0.005959451198577881, -0.011010980233550072, 0.05383145809173584, -0.005257581826299429, 0.018712226301431656, -0.01159694418311119, 0.0044398074969649315, 0.008950445801019669, 0.012073442339897156, 0.008602729998528957, 0.0030247997492551804, -0.011217033490538597, -0.001978434855118394, -0.02224089205265045, 0.009388308972120285, -0.005827448330819607, 0.030083799734711647, 0.0035383235663175583, -0.003396661952137947, -0.00838379841297865, -0.029620179906487465, -0.016072167083621025, 0.0063232

laten we eens kijken of we de meest relevante pagina kunnen vinden om de vraag te beantwoorden.

* We gaan een vector maken van de vraag
* we gaan voor iedere pagina een vector maken
* We gaan de meest relevante pagina selecteren en in de prompt meegeven

In [60]:
#lees de pagina's van de website
page1 = open("../docs/page-0.md", "r").read()
page2 = open("../docs/page-1.md", "r").read()
page3 = open("../docs/page-2.md", "r").read()

pages = [page1, page2, page3]

myDataQuestion = "Wat is er speciaal aan de vrijdag voor de opleiding HBO-ICT in Heerlen?"

pagesListEmbedding = []
for p in pages:
    response = openai_client.embeddings.create(input = p, model=embeddings_deployment_name).data[0].embedding
    pagesListEmbedding.append(response)

#create an embedding for the word we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#calculate the dot product between the word we want to compare and each word in the list
dotProductList = []
for p in pagesListEmbedding:
    dotProductList.append(dotproduct(questionEmbedding, p))

#sort the dotProductList
sortedDotProductList = sorted(dotProductList, reverse=True)

#find the index of the words with the highest dot product
indexList = []
for i in sortedDotProductList:
    indexList.append(dotProductList.index(i))

#augment the prompt with the most relevant page
myDataResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": myDataQuestion
        },
        {
            "role": "system",
            "content" : f"""
                Je bent een behulpzame assistent
                Je geeft alleen antwoord op basis van de data die je hebt gekregen
                Indien je het antwoord niet kunt vinden, 
                zeg je dat je het antwoord niet weet
                
                ## data
                {pages[indexList[0]]}

                """
        }
    ]
)


lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

print("----")
print("Hier wat info over de tokens die je hebt gebruikt:")
print(myDataResponse.usage)



Op vrijdag is er bij de HBO-ICT opleiding in Heerlen een speciale focus op
evaluatie en professionele vaardigheden. Dit is wat de vrijdag speciaal maakt:
- **Evaluatiedag**: Vrijdag is de dag waarop studenten evalueren wat ze
gedurende de week hebben geleerd en bereikt. Ze kijken of ze alle stof
beheersen, of alle taken af zijn en hoe de samenwerking in hun groep is
verlopen. - **Professional Skills**: Op vrijdag ligt de nadruk ook op de
ontwikkeling van 'professional skills'. Studenten presenteren wat ze hebben
gedaan en krijgen feedback op hun professionele vaardigheden. -
**Gastsprekers**: Soms komt er een ervaren ICT-professional langs om te
vertellen over zijn of haar werk, wat studenten inzicht geeft in de praktijk van
de ICT-sector.  Deze combinatie van activiteiten maakt de vrijdag tot een
belangrijke dag in de weekstructuur van de HBO-ICT opleiding in Heerlen, waarbij
reflectie op de eigen voortgang en professionele ontwikkeling centraal staan.
----
Hier wat info over de token

Dit is al een stuk beter!

Maar hebben we het hele document wel nodig?

In [61]:
# create an array of files in the chunks directory
files = os.listdir("../chunks")
pages = []

for file in files:
    page = open(f"../chunks/{file}", "r").read()
    pages.append(page)

pagesListEmbedding = []
for p in pages:
    response = openai_client.embeddings.create(input = p, model=embeddings_deployment_name).data[0].embedding
    pagesListEmbedding.append(response)

#create an embedding for the word we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#calculate the dot product between the word we want to compare and each word in the list
dotProductList = []
for p in pagesListEmbedding:
    dotProductList.append(dotproduct(questionEmbedding, p))

#sort the dotProductList
sortedDotProductList = sorted(dotProductList, reverse=True)

#find the index of the words with the highest dot product
indexList = []
for i in sortedDotProductList:
    indexList.append(dotProductList.index(i))

#augment the prompt with the most relevant page
myDataResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": myDataQuestion
        },
        {
            "role": "system",
            "content" : f"""
                Je bent een behulpzame assistent
                Je geeft alleen antwoord op basis van de data die je hebt gekregen
                Indien je het antwoord niet kunt vinden, 
                zeg je dat je het antwoord niet weet
                
                ## data
                {pages[indexList[0]]}

                """
        }
    ]
)


lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

print("----")
print("Hier wat info over de tokens die je hebt gebruikt:")
print(myDataResponse.usage)



Het lijkt erop dat de vrijdag voor de opleiding HBO-ICT in Heerlen speciaal is
omdat het een gelegenheid biedt voor potentiële studenten om te proefstuderen.
Tijdens zo'n dag kunnen geïnteresseerden ervaren hoe het is om te studeren aan
deze opleiding, door deel te nemen aan speciaal voorbereide opdrachten, de
lokalen te verkennen en kennis te maken met huidige studenten en docenten. Dit
is een kans om een beter beeld te krijgen van de opleiding en de sfeer op de
campus.
----
Hier wat info over de tokens die je hebt gebruikt:
CompletionUsage(completion_tokens=142, prompt_tokens=195, total_tokens=337)


Weer een verbetering! Minder tokens en toch een antwoord!

Embeddings berekenen kost echter ook tokens

In [62]:
response = openai_client.embeddings.create(input = pages[indexList[0]], model=embeddings_deployment_name)
response.usage

Usage(prompt_tokens=98, total_tokens=98)

Vectoren berekenen hoeven we niet iedere keer te doen.

We kunnen de vectoren ook opslaan en hergebruiken.

Hier hebben we een vector database voor nodig. 

Er zijn verschillende vector databases. 

We kiezen hier voor MongoDb in Azure. We gebruiken Cosine Simularity als algoritme.

In [63]:
databaseName = "hszuyd"
collectionName = "webpagecollection"

mongo_client = pymongo.MongoClient(mongo_conn)
db = mongo_client[databaseName]
collection = db[collectionName]

if collectionName not in db.list_collection_names():
    db.create_collection(collectionName)
else:
    #delete all documents in the collection so we can rerun this script
    collection.delete_many({})


db.command({
  'createIndexes': f'{collectionName}',
  'indexes': [
    {
      'name': 'VectorSearchIndex',
      'key': {
        "contentVector": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 1,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
})

files = os.listdir("../chunks")
pages = []
filesUploaded = 0
for file in files:
    page = open(f"../chunks/{file}", "r").read()
    response = openai_client.embeddings.create(input = page, model=embeddings_deployment_name).data[0].embedding
    
    filetoStore = {
        "filename": file,
        "content": page,
        "contentVector": response
    }

    collection.insert_one(filetoStore)
    filesUploaded += 1

print(f"{filesUploaded} bestanden en vectoren opgeslagen in de database")


23 bestanden en vectoren opgeslagen in de database


We kunnen nu een database query doen met de vector van de vraag als input.

De output is een lijst met relevante documenten die in de database zijn opgeslagen.

In [64]:
def vector_search(vector, num_results):
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": vector,
                    "path": "contentVector",
                    "k": num_results 
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

myDataQuestion = "Wat is er speciaal aan de vrijdag voor de opleiding HBO-ICT in Heerlen?"
#create an embedding for the question we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#zoek 3 relevante pagina's op basis van de vraag
results = vector_search(questionEmbedding, 3)

print("De top 3 resultaten zijn: ")
for result in results:
    print(result["document"]["filename"] + " - " + str(result["similarityScore"]))


De top 3 resultaten zijn: 
hbo-ict-16.md - 0.8858831780800895
hbo-ict-6.md - 0.883809857595556
hbo-ict-1.md - 0.881071879735986
